In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

# 사용할 디바이스 설정 (GPU가 있다면 GPU 사용)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# # 랜덤 시드 고정 (재현성을 위해)
# torch.manual_seed(0)
# if device == 'cuda':
#     torch.cuda.manual_seed_all(0)

# 데이터 변환 정의 (Tensor 변환 후 정규화)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# MNIST 데이터셋 로드
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# 하이퍼파라미터 설정
batch_size = 64
learning_rate = 0.001
num_epochs = 10

# 데이터 로더 정의
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Using device: cuda


100%|██████████| 9.91M/9.91M [00:00<00:00, 17.2MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 458kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.36MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.07MB/s]


In [2]:
# CNN 모델 정의
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        # 첫 번째 합성곱 계층
        # 입력 채널: 1 (MNIST는 흑백 이미지)
        # 출력 채널: 32
        # 커널 크기: 3x3
        # 패딩: 1 (출력 크기 유지)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)  # 2x2 최대 풀링 (특성 맵 크기 절반 감소)
        )

        # 두 번째 합성곱 계층
        # 입력 채널: 32
        # 출력 채널: 64
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 완전 연결층 (FC 레이어) - layer3 제거 후 수정
        # Conv 계층을 지나면 7x7x64 크기의 텐서가 됨 -> 이를 Flatten하여 3136 차원 벡터로 변환
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(7 * 7 * 64, 128),  # FC 레이어 1
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),  # 과적합 방지를 위한 드롭아웃
            torch.nn.Linear(128, 10)  # FC 레이어 2 (출력: 10개 클래스)
        )
        torch.nn.init.xavier_uniform_(self.fc[0].weight)  # Xavier 초기화 적용

    def forward(self, x):
        x = self.layer1(x)  # 첫 번째 합성곱 계층 통과
        x = self.layer2(x)  # 두 번째 합성곱 계층 통과
        x = x.view(x.size(0), -1)  # Flatten (FC 입력을 위해)
        x = self.fc(x)  # 완전 연결층 통과
        return x

In [3]:
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [4]:
# 학습 및 평가 루프
best_accuracy = 0
for epoch in range(num_epochs):
    model.train()
    avg_cost = 0

    for X, Y in train_loader:
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost += cost / len(train_loader)

    # 학습률 업데이트
    scheduler.step()

    # 테스트 정확도 평가
    model.eval()
    test_accuracy = 0
    with torch.no_grad():
        for X_test, Y_test in test_loader:
            X_test, Y_test = X_test.to(device), Y_test.to(device)
            prediction = model(X_test)
            test_accuracy += (torch.argmax(prediction, 1) == Y_test).float().mean()
    test_accuracy /= len(test_loader)

    # 최고 정확도 업데이트
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_cost:.4f}, Test Accuracy: {test_accuracy*100:.2f}%")

# 최종 최고 정확도 출력
print(f"Best Test Accuracy: {best_accuracy*100:.2f}%")


Epoch [1/10], Loss: 0.7969, Test Accuracy: 98.42%
Epoch [2/10], Loss: 0.6849, Test Accuracy: 98.87%
Epoch [3/10], Loss: 0.6643, Test Accuracy: 98.94%
Epoch [4/10], Loss: 0.6516, Test Accuracy: 99.10%
Epoch [5/10], Loss: 0.6409, Test Accuracy: 99.19%
Epoch [6/10], Loss: 0.6212, Test Accuracy: 99.32%
Epoch [7/10], Loss: 0.6132, Test Accuracy: 99.31%
Epoch [8/10], Loss: 0.6072, Test Accuracy: 99.26%
Epoch [9/10], Loss: 0.5993, Test Accuracy: 99.26%
Epoch [10/10], Loss: 0.5926, Test Accuracy: 99.28%
Best Test Accuracy: 99.32%
